### 19_06 The readers-writers problem

Consider an object *s* which is read from and written to by many threads.  (For example, *s* could be the cache from Problem 19.1 on Page 291.)  You need to ensure that no thread may access *s* for reading or writing while another thread is writing to s.  (Two or more readers may access s at the same time.)

One way to achieve this is by protecting *s* with a mutex that ensures that two threads cannot access *s* at the same time.  However, this solution is suboptimal because it is possible that a reader *R1* has locked *s* and another reader *R2* wants to access *s*.  Reader *R2* does not have to wait until *R1* is done reading, *R2* should start reading right away.  

This motivates the first readers-writers problem:  protect s with the added constraint that no reader is to be kept waiting if s is currently opened for reading.

Implement a synchronization mechanism for the first readers-writers problem.

**Hint**: Track the number of readers

### Remarks

It seems like there should be a reader counter that increments for every reader that wishes to read.  This reader counter guards the data from writers, and must be zero before the writer can do its work.  You may wish to make it such that a queued up writer blocks new readers.  I also think that since writers are important and we do not want *dirty* data, that writers should have a mutex that prevents both readers and writers.  Let's debug the book solution with PyCharm.

In [1]:
import time
import random
import threading


def do_something_else():
    time.sleep(random.random())

# LR and LW are class attributes in the RW class.
# They serve as read and write locks. The integer
# variable read_count in RW tracks the number of readers.
class RW:
    data = 0
    LR = threading.Condition()
    read_count = 0
    LW = threading.Lock()


class Reader(threading.Thread):
    """The Reader class prints data from the RW class while running.
    It manipulates the RW.read_count by incrementing before a read
    and by decrementing after a read."""
    def __init__(self, name):
        super().__init__(name=name, daemon=True)

    def run(self):
        while True:
            with RW.LR:
                RW.read_count += 1

            print('Reader', self.name, 'is about to read')
            print(RW.data)
            with RW.LR:
                RW.read_count -= 1
                RW.LR.notify()
            do_something_else()


class Writer(threading.Thread):
    """The Writer class gets the RW.LW Lock and updates the RW.data"""
    def __init__(self, name):
        super().__init__(name=name, daemon=True)

    def run(self):
        while True:
            with RW.LW:
                done = False
                while not done:
                    with RW.LR:
                        if RW.read_count == 0:
                            # @exclude
                            print('Writer', self.name, 'is about to write')
                            # @include
                            RW.data += 1
                            done = True
                        else:
                            # use wait/notify to avoid busy waiting
                            while RW.read_count != 0:
                                RW.LR.wait()
            do_something_else()


# This merely creates and starts readers and writers, it then sleeps for 10 seconds before terminating.
def main():
    r0 = Reader('r0')
    r1 = Reader('r1')
    w0 = Writer('w0')
    w1 = Writer('w1')
    r0.start()
    r1.start()
    w0.start()
    w1.start()
    time.sleep(10)


if __name__ == '__main__':
    main()

Reader r0 is about to read
0
Reader r1 is about to read
0
Writer w0 is about to write
Writer w1 is about to write
Writer w1 is about to write
Reader r0 is about to read
3
Writer w0 is about to write
Writer w0 is about to write
Writer w1 is about to write
Reader r1 is about to read
6
Reader r0 is about to read
6
Writer w0 is about to write
Reader r1 is about to read
7
Reader r0 is about to read
7
Writer w1 is about to write
Reader r1 is about to read
8
Reader r1 is about to read
8
Reader r0 is about to read
8
Reader r0 is about to read
8
Writer w0 is about to write
Writer w1 is about to write
Writer w0 is about to write
Reader r0 is about to read
11
Reader r1 is about to read
11
Writer w0 is about to write
Reader r0 is about to read
12
Writer w1 is about to write
Reader r0 is about to read
13
Reader r0 is about to read
13
Reader r1 is about to read
13
Reader r1 is about to read
13
Writer w0 is about to write
Writer w1 is about to write
Writer w0 is about to write
Reader r1 is about to r